# Experiment 1: Random Sparse Networks versus Noisy Labels

In this first experiment, we aim to determine whether sparse networks show any potential for being robust against noisy labels in comparison to dense networks. 

In [1]:
# loading CIFAR10/100 datasets
from dataloaders.cifar import CifarDataloader
from train import *
from pathlib import Path

In [2]:
# Directory for downloading and reading training data from
datapath = Path('./data')
if not datapath.exists():
    datapath.mkdir(exist_ok=True)
# Directory for saving model weights during training
weightsDir = Path('models/weights')
weightsDir.mkdir(exist_ok=True)
# Directory for saving results during training
resultsDir = Path('results')
resultsDir.mkdir(exist_ok=True)

In [3]:
def run(dataset, noise_rate, noise_mode, sparsity, batch_size, 
        datapath, noise_file, weightFileName, repeats):
    """
    A wrapper function to run the training loop (train, evaluate, test) with specified parameters.
    """
    # intialise device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # same optimisation and epoch schedule used
    epochs = 200
    
    # running averages
    avgTrainAcc, avgTrainLoss = 0,0
    avgValAcc, avgValLoss = 0,0
    avgTestAcc, avgTestLoss = 0,0
    
    
    for r in range(repeats):
        f = open(resultsDir / Path(weightFileName + f'_{r}.txt'), 'w') # save results for this run

        # get data loaders for training, validation, and test sets
        # different seed used for every repeat to split the training into train + valid set
        dataLoader = CifarDataloader(dataset=dataset, noise_rate=noise_rate, noise_mode=noise_mode, 
                                     batch_size=batch_size, datapath=datapath, noise_file = noise_file, valid_seed = r)
        train_loader = dataLoader.trainLoader
        val_loader = dataLoader.validLoader
        test_loader = dataLoader.testLoader
        n = len(train_loader.dataset)

        # initialise model and learning rate schedule
        model, optimizer = get_model(dataset, lr=0.1, sparsity=sparsity)
        schedule = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2) # need to adjust this schedule to match a paper
        model = model.to(device)

        # save the model for best validaiton loss throughout training    
        best_epoch, best_val_loss = 0, np.inf
        train_loss_list, train_acc_list = [], []
        val_loss_list, val_acc_list = [], []

        for epoch in range(epochs):

            start = time.time()
            print("Epoch: {}".format(epoch+1))
            f.write("Epoch: {}\n".format(epoch+1))

            train_loss, train_acc = train(model, optimizer, train_loader, device)
            train_loss /= n
            print("Train Loss: {:.6f}, Acc: {:.6f}".format(train_loss, train_acc))
            f.write("Train Loss: {:.6f}, Acc: {:.6f}\n".format(train_loss, train_acc))

            val_loss, val_acc = evaluate(model, val_loader, device)
            print("Val Loss: {:.6f},  Acc: {:.6f}".format(val_loss, val_acc))
            f.write("Val Loss: {:.6f},  Acc: {:.6f}\n".format(val_loss, val_acc))

            if val_loss < best_val_loss:
                best_epoch = epoch
                best_val_loss = val_loss
                best_train_loss = train_loss
                torch.save(model.state_dict(), weightsDir / Path(f'{weightFileName}_best_forward_{r}.pkl'))

            schedule.step()

            train_loss_list.append(train_loss)
            train_acc_list.append(train_acc)
            val_loss_list.append(val_loss)
            val_acc_list.append(val_acc)

            end = time.time()
            print("Took {:.2f} minutes.\n".format((end - start) / 60))
            f.write("Took {:.2f} minutes.\n".format((end - start) / 60))

        print("Best epoch:", best_epoch+1)
        f.write(f"Best epoch: {best_epoch+1}\n")
        print("Final train loss: {:.4f}".format(train_loss_list[best_epoch]))
        f.write("Final train loss: {:.4f}\n".format(train_loss_list[best_epoch]))
        print("Final train accuracy: {:.4f}\n".format(train_acc_list[best_epoch]))
        f.write("Final train accuracy: {:.4f}\n".format(train_acc_list[best_epoch]))
        print("Final val loss: {:.4f}".format(val_loss_list[best_epoch]))
        f.write("Final val loss: {:.4f}\n".format(val_loss_list[best_epoch]))
        print("Final val accuracy: {:.4f}\n".format(val_acc_list[best_epoch]))
        f.write("Final val accuracy: {:.4f}\n".format(val_acc_list[best_epoch]))

        # load best model saved during training
        model.load_state_dict(torch.load(weightsDir / Path(f'{weightFileName}_best_forward_{r}.pkl'), map_location=device))
        test_loss, test_acc = evaluate(model, test_loader, device)
        print("Test accuracy: {:.4f}".format(test_acc))
        f.write("Test accuracy: {:.4f}\n".format(test_acc))
        
        # save average results
        avgTrainAcc += train_acc_list[best_epoch]
        avgTrainLoss += train_loss_list[best_epoch]
        avgValAcc += val_acc_list[best_epoch]
        avgValLoss += val_loss_list[best_epoch]
        avgTestAcc += test_acc
        avgTestLoss += test_loss

        f.close()
        
    # save average results for this run    
    with open(resultsDir / Path(weightFileName + f'_avg.txt'), 'w') as f:
        avgTrainAcc /= repeats
        avgTrainLoss /= repeats
        avgValAcc /= repeats
        avgValLoss /= repeats
        avgTestAcc /= repeats
        avgTestLoss /= repeats
        
        f.write(f"Repeats: {repeats}\n")
        f.write(f"Average Training Accuracy: {avgTrainAcc}\n")
        f.write(f"Average Training Loss: {avgTrainLoss}\n")
        f.write(f"Average Validation Accuracy: {avgValAcc}\n")
        f.write(f"Average Validation Loss: {avgValLoss}\n")
        f.write(f"Average Test Accuracy: {avgTestAcc}\n")
        f.write(f"Average Test Loss: {avgTestLoss}\n")

Experiments In this Notebook:
* Dense no noise, Cifar10 with 3-fold validation
* Dense 20% noise, Cifar10 with 3-fold validation
* Sparse no noise, Cifar10 with 3-fold validation
* Sparse 20% noise, Cifar10 with 3-fold validation
* Dense no noise, Cifar100 with 3-fold validation
* Dense 20% noise, Cifar100 with 3-fold validation
* Sparse no noise, Cifar100 with 3-fold validation
* Sparse 20% noise, Cifar100 with 3-fold validation

In [8]:
# Settings for training run
dataset = "cifar10"
noise_rate = 0.0
noise_mode="sym"
batch_size=128
datapath=datapath
noise_file = "NoNoiseCifar10"
sparsity = 0
weightFileName = f"R18_Cifar10_{int(noise_rate*100)}pct_{noise_mode}"
repeats = 3

# function to run training and evaluation loop
run(dataset, noise_rate, noise_mode, sparsity, batch_size, 
    datapath, noise_file, weightFileName, repeats)

Files already downloaded and verified
Epoch: 1
Train Loss: 0.000047, Acc: 0.316725
Val Loss: 1.610681,  Acc: 0.398300
Took 0.34 minutes.

Epoch: 2
Train Loss: 0.000037, Acc: 0.460425
Val Loss: 1.475720,  Acc: 0.449100
Took 0.34 minutes.

Epoch: 3
Train Loss: 0.000031, Acc: 0.548850
Val Loss: 1.360492,  Acc: 0.527600
Took 0.34 minutes.

Epoch: 4
Train Loss: 0.000026, Acc: 0.624300
Val Loss: 1.335873,  Acc: 0.563200
Took 0.34 minutes.

Epoch: 5
Train Loss: 0.000022, Acc: 0.682025
Val Loss: 1.018218,  Acc: 0.650900
Took 0.34 minutes.

Epoch: 6
Train Loss: 0.000019, Acc: 0.729025
Val Loss: 1.072719,  Acc: 0.656400
Took 0.34 minutes.

Epoch: 7
Train Loss: 0.000017, Acc: 0.764200
Val Loss: 0.960121,  Acc: 0.689200
Took 0.34 minutes.

Epoch: 8
Train Loss: 0.000015, Acc: 0.790450
Val Loss: 0.726246,  Acc: 0.742300
Took 0.34 minutes.

Epoch: 9
Train Loss: 0.000014, Acc: 0.805350
Val Loss: 0.669113,  Acc: 0.775900
Took 0.34 minutes.

Epoch: 10
Train Loss: 0.000013, Acc: 0.822150
Val Loss: 0.6977

In [ ]:
# Settings for training run
dataset = "cifar10"
noise_rate = 0.2
noise_mode="sym"
batch_size=128
datapath=datapath
noise_file = "20SymNoiseCifar10"
sparsity = 0
weightFileName = f"R18_Cifar10_{int(noise_rate*100)}pct_{noise_mode}"
repeats = 3

# function to run training and evaluation loop
run(dataset, noise_rate, noise_mode, sparsity, batch_size, 
    datapath, noise_file, weightFileName, repeats)

Files already downloaded and verified
saving noisy labels to data/noisyLabels/20SymNoiseCifar10 ...
Epoch: 1
Train Loss: 0.000055, Acc: 0.229850
Val Loss: 2.055798,  Acc: 0.251000
Took 0.45 minutes.

Epoch: 2
Train Loss: 0.000047, Acc: 0.335475
Val Loss: 1.903168,  Acc: 0.339200
Took 0.34 minutes.

Epoch: 3
Train Loss: 0.000045, Acc: 0.394075
Val Loss: 1.858922,  Acc: 0.357100
Took 0.34 minutes.

Epoch: 4
Train Loss: 0.000042, Acc: 0.443100
Val Loss: 1.694291,  Acc: 0.445000
Took 0.34 minutes.

Epoch: 5
Train Loss: 0.000040, Acc: 0.496125
Val Loss: 1.652903,  Acc: 0.467600
Took 0.34 minutes.

Epoch: 6
Train Loss: 0.000038, Acc: 0.521725
Val Loss: 1.547277,  Acc: 0.512500
Took 0.34 minutes.

Epoch: 7
Train Loss: 0.000037, Acc: 0.551675
Val Loss: 1.485771,  Acc: 0.544900
Took 0.34 minutes.

Epoch: 8
Train Loss: 0.000036, Acc: 0.567000
Val Loss: 1.525417,  Acc: 0.538000
Took 0.34 minutes.

Epoch: 9
Train Loss: 0.000035, Acc: 0.587175
Val Loss: 1.512420,  Acc: 0.542800
Took 0.34 minutes.



In [4]:
# Settings for training run
dataset = "cifar10"
noise_rate = 0.0
noise_mode="sym"
batch_size=128
datapath=datapath
noise_file = "NoNoiseCifar10"
sparsity = 0.8
weightFileName = f"R18_20SparseCifar10_{int(noise_rate*100)}pct_{noise_mode}"
repeats = 3

# function to run training and evaluation loop
run(dataset, noise_rate, noise_mode, sparsity, batch_size, 
    datapath, noise_file, weightFileName, repeats)

Files already downloaded and verified
Epoch: 1
Train Loss: 0.000052, Acc: 0.256100
Val Loss: 1.736105,  Acc: 0.346700
Took 0.41 minutes.

Epoch: 2
Train Loss: 0.000039, Acc: 0.412450
Val Loss: 1.656876,  Acc: 0.419500
Took 0.34 minutes.

Epoch: 3
Train Loss: 0.000033, Acc: 0.510400
Val Loss: 1.301855,  Acc: 0.528200
Took 0.34 minutes.

Epoch: 4
Train Loss: 0.000028, Acc: 0.601700
Val Loss: 1.646860,  Acc: 0.477200
Took 0.34 minutes.

Epoch: 5
Train Loss: 0.000024, Acc: 0.652925
Val Loss: 0.961600,  Acc: 0.660400
Took 0.34 minutes.

Epoch: 6
Train Loss: 0.000021, Acc: 0.699950
Val Loss: 1.319025,  Acc: 0.582900
Took 0.34 minutes.

Epoch: 7
Train Loss: 0.000019, Acc: 0.735275
Val Loss: 0.899442,  Acc: 0.685700
Took 0.35 minutes.

Epoch: 8
Train Loss: 0.000017, Acc: 0.768600
Val Loss: 0.784734,  Acc: 0.727700
Took 0.35 minutes.

Epoch: 9
Train Loss: 0.000015, Acc: 0.788850
Val Loss: 0.803607,  Acc: 0.729800
Took 0.34 minutes.

Epoch: 10
Train Loss: 0.000014, Acc: 0.810775
Val Loss: 0.6830

In [4]:
# Settings for training run
dataset = "cifar10"
noise_rate = 0.2
noise_mode="sym"
batch_size=128
datapath=datapath
noise_file = "20SymNoiseCifar10"
sparsity = 0.8
weightFileName = f"R18_20SparseCifar10_{int(noise_rate*100)}pct_{noise_mode}"
repeats = 3

# function to run training and evaluation loop
run(dataset, noise_rate, noise_mode, sparsity, batch_size, 
    datapath, noise_file, weightFileName, repeats)

Files already downloaded and verified
Epoch: 1
Train Loss: 0.000058, Acc: 0.198675
Val Loss: 2.008331,  Acc: 0.281600
Took 0.38 minutes.

Epoch: 2
Train Loss: 0.000048, Acc: 0.314700
Val Loss: 1.916070,  Acc: 0.331000
Took 0.34 minutes.

Epoch: 3
Train Loss: 0.000045, Acc: 0.381875
Val Loss: 1.787776,  Acc: 0.395000
Took 0.34 minutes.

Epoch: 4
Train Loss: 0.000043, Acc: 0.426000
Val Loss: 1.807379,  Acc: 0.398600
Took 0.34 minutes.

Epoch: 5
Train Loss: 0.000041, Acc: 0.474125
Val Loss: 1.697588,  Acc: 0.440400
Took 0.35 minutes.

Epoch: 6
Train Loss: 0.000039, Acc: 0.512025
Val Loss: 1.574236,  Acc: 0.511200
Took 0.35 minutes.

Epoch: 7
Train Loss: 0.000037, Acc: 0.545125
Val Loss: 1.554722,  Acc: 0.525000
Took 0.35 minutes.

Epoch: 8
Train Loss: 0.000036, Acc: 0.570075
Val Loss: 1.453726,  Acc: 0.566300
Took 0.35 minutes.

Epoch: 9
Train Loss: 0.000035, Acc: 0.590200
Val Loss: 1.435649,  Acc: 0.574100
Took 0.35 minutes.

Epoch: 10
Train Loss: 0.000034, Acc: 0.610650
Val Loss: 1.3665

In [ ]:
# Settings for training run
dataset = "cifar100"
noise_rate = 0.0
noise_mode="sym"
batch_size=128
datapath=datapath
noise_file = "NoNoiseCifar100"
sparsity = 0
weightFileName = f"R34_Cifar100_{int(noise_rate*100)}pct_{noise_mode}"
repeats = 3

# function to run training and evaluation loop
run(dataset, noise_rate, noise_mode, sparsity, batch_size, 
    datapath, noise_file, weightFileName, repeats)

Files already downloaded and verified
saving noisy labels to data/noisyLabels/NoNoiseCifar100 ...
Epoch: 1
Train Loss: 0.000106, Acc: 0.056075
Val Loss: 3.897211,  Acc: 0.090600
Took 0.67 minutes.

Epoch: 2
Train Loss: 0.000094, Acc: 0.109950
Val Loss: 3.787276,  Acc: 0.114800
Took 0.56 minutes.

Epoch: 3
Train Loss: 0.000087, Acc: 0.158250
Val Loss: 3.489764,  Acc: 0.161800
Took 0.57 minutes.

Epoch: 4
Train Loss: 0.000079, Acc: 0.213300
Val Loss: 3.077896,  Acc: 0.232800
Took 0.57 minutes.

Epoch: 5
Train Loss: 0.000073, Acc: 0.256625
Val Loss: 2.972237,  Acc: 0.267600
Took 0.57 minutes.

Epoch: 6
Train Loss: 0.000067, Acc: 0.305975
Val Loss: 2.854808,  Acc: 0.283800
Took 0.57 minutes.

Epoch: 7
Train Loss: 0.000062, Acc: 0.354275
Val Loss: 2.832361,  Acc: 0.297300
Took 0.57 minutes.

Epoch: 8
Train Loss: 0.000056, Acc: 0.396675
Val Loss: 2.255657,  Acc: 0.401800
Took 0.57 minutes.

Epoch: 9
Train Loss: 0.000051, Acc: 0.443475
Val Loss: 2.192515,  Acc: 0.416200
Took 0.57 minutes.

Ep

In [4]:
# Settings for training run
dataset = "cifar100"
noise_rate = 0.2
noise_mode="sym"
batch_size=128
datapath=datapath
noise_file = "20SymCifar100"
sparsity = 0
weightFileName = f"R34_Cifar100_{int(noise_rate*100)}pct_{noise_mode}"
repeats = 3

# function to run training and evaluation loop
run(dataset, noise_rate, noise_mode, sparsity, batch_size, 
    datapath, noise_file, weightFileName, repeats)

Files already downloaded and verified
saving noisy labels to data/noisyLabels/20SymCifar100 ...
Epoch: 1
Train Loss: 0.000112, Acc: 0.039100
Val Loss: 4.231835,  Acc: 0.058800
Took 0.66 minutes.

Epoch: 2
Train Loss: 0.000103, Acc: 0.079775
Val Loss: 4.093821,  Acc: 0.078700
Took 0.59 minutes.

Epoch: 3
Train Loss: 0.000100, Acc: 0.104150
Val Loss: 3.926890,  Acc: 0.122100
Took 0.59 minutes.

Epoch: 4
Train Loss: 0.000096, Acc: 0.136900
Val Loss: 3.785195,  Acc: 0.155300
Took 0.59 minutes.

Epoch: 5
Train Loss: 0.000093, Acc: 0.168250
Val Loss: 3.721381,  Acc: 0.168800
Took 0.60 minutes.

Epoch: 6
Train Loss: 0.000090, Acc: 0.194100
Val Loss: 3.561255,  Acc: 0.206300
Took 0.59 minutes.

Epoch: 7
Train Loss: 0.000086, Acc: 0.225525
Val Loss: 3.607320,  Acc: 0.202700
Took 0.57 minutes.

Epoch: 8
Train Loss: 0.000083, Acc: 0.254250
Val Loss: 3.455630,  Acc: 0.245800
Took 0.57 minutes.

Epoch: 9
Train Loss: 0.000080, Acc: 0.282700
Val Loss: 3.379395,  Acc: 0.259000
Took 0.59 minutes.

Epoc

In [ ]:
# Settings for training run
dataset = "cifar100"
noise_rate = 0
noise_mode="sym"
batch_size=128
datapath=datapath
noise_file = "NoNoiseCifar100"
sparsity = 0.8
weightFileName = f"R34_20SparseCifar100_{int(noise_rate*100)}pct_{noise_mode}"
repeats = 3

# function to run training and evaluation loop
run(dataset, noise_rate, noise_mode, sparsity, batch_size, 
    datapath, noise_file, weightFileName, repeats)

Files already downloaded and verified
Epoch: 1
Train Loss: 0.000106, Acc: 0.058100
Val Loss: 3.937102,  Acc: 0.085100
Took 0.61 minutes.

Epoch: 2
Train Loss: 0.000093, Acc: 0.115850
Val Loss: 3.698871,  Acc: 0.133200
Took 0.67 minutes.

Epoch: 3
Train Loss: 0.000085, Acc: 0.171000
Val Loss: 3.369182,  Acc: 0.176800
Took 0.67 minutes.

Epoch: 4
Train Loss: 0.000078, Acc: 0.226325
Val Loss: 3.021651,  Acc: 0.240700
Took 0.68 minutes.

Epoch: 5
Train Loss: 0.000070, Acc: 0.283125
Val Loss: 2.839401,  Acc: 0.286300
Took 0.67 minutes.

Epoch: 6
Train Loss: 0.000064, Acc: 0.331025
Val Loss: 2.828771,  Acc: 0.293400
Took 0.66 minutes.

Epoch: 7
Train Loss: 0.000058, Acc: 0.377300
Val Loss: 2.452519,  Acc: 0.365300
Took 0.66 minutes.

Epoch: 8
Train Loss: 0.000053, Acc: 0.424850
Val Loss: 2.320320,  Acc: 0.392100
Took 0.68 minutes.

Epoch: 9
Train Loss: 0.000049, Acc: 0.458900
Val Loss: 2.297387,  Acc: 0.405700
Took 0.66 minutes.

Epoch: 10
Train Loss: 0.000046, Acc: 0.492125
Val Loss: 2.1425

In [ ]:
import numpy as np
import time

# Directory for saving model weights during training
weightsDir = Path('models/weights')
weightsDir.mkdir(exist_ok=True)
resultsDir = Path('results')
resultsDir.mkdir(exist_ok=True)

# Settings for training run
dataset = "cifar100"
noise_rate = 0.2
noise_mode="sym"
batch_size=128
datapath=datapath
noise_file = "dense20pctSymCifar100"
epochs = 200
weightFileName = f"R34_15Sprase_Cifar100_{int(noise_rate*100)}pct_{noise_mode}"
sparsity = 0.85

f = open(resultsDir / Path(weightFileName + '.txt'), 'w')

# intialise device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# get data loaders for training, validation, and test sets
dataLoader = CifarDataloader(dataset=dataset, noise_rate=noise_rate, noise_mode=noise_mode, batch_size=batch_size, datapath=datapath, noise_file = noise_file)
train_loader = dataLoader.trainLoader
val_loader = dataLoader.validLoader
test_loader = dataLoader.testLoader
n = len(train_loader.dataset)

# initialise model and learning rate schedule
model, optimizer = get_model(dataset, lr=0.1)
schedule = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2) # need to adjust this schedule to match a paper
model = model.to(device)

# save the model for best validaiton loss throughout training    
best_epoch, best_val_loss = 0, np.inf
train_loss_list, train_acc_list = [], []
val_loss_list, val_acc_list = [], []

for epoch in range(epochs):

    start = time.time()
    print("Epoch: {}".format(epoch+1))
    f.write("Epoch: {}\n".format(epoch+1))

    train_loss, train_acc = train(model, optimizer, train_loader, device)
    train_loss /= n
    print("Train Loss: {:.6f}, Acc: {:.6f}".format(train_loss, train_acc))
    f.write("Train Loss: {:.6f}, Acc: {:.6f}\n".format(train_loss, train_acc))

    val_loss, val_acc = evaluate(model, val_loader, device)
    print("Val Loss: {:.6f},  Acc: {:.6f}".format(val_loss, val_acc))
    f.write("Val Loss: {:.6f},  Acc: {:.6f}\n".format(val_loss, val_acc))

    if val_loss < best_val_loss:
        best_epoch = epoch
        best_val_loss = val_loss
        best_train_loss = train_loss
        torch.save(model.state_dict(), weightsDir / Path(f'{weightFileName}_best_forward.pkl'))

    schedule.step()

    train_loss_list.append(train_loss)
    train_acc_list.append(train_acc)
    val_loss_list.append(val_loss)
    val_acc_list.append(val_acc)

    end = time.time()
    print("Took {:.2f} minutes.\n".format((end - start) / 60))
    f.write("Took {:.2f} minutes.\n".format((end - start) / 60))

print("Best epoch:", best_epoch+1)
f.write(f"Best epoch: {best_epoch+1}\n")
print("Final train loss: {:.4f}".format(train_loss_list[best_epoch]))
f.write("Final train loss: {:.4f}\n".format(train_loss_list[best_epoch]))
print("Final train accuracy: {:.4f}\n".format(train_acc_list[best_epoch]))
f.write("Final train accuracy: {:.4f}\n".format(train_acc_list[best_epoch]))
print("Final val loss: {:.4f}".format(val_loss_list[best_epoch]))
f.write("Final val loss: {:.4f}\n".format(val_loss_list[best_epoch]))
print("Final val accuracy: {:.4f}\n".format(val_acc_list[best_epoch]))
f.write("Final val accuracy: {:.4f}\n".format(val_acc_list[best_epoch]))

# load best model saved during training
model.load_state_dict(torch.load(weightsDir / Path(f'{weightFileName}_best_forward.pkl'), map_location=device))
test_loss, test_acc = evaluate(model, test_loader, device)
print("Test accuracy: {:.4f}".format(test_acc))
f.write("Test accuracy: {:.4f}\n".format(test_acc))

f.close()

Files already downloaded and verified
Epoch: 1
Train Loss: 0.000112, Acc: 0.040200
Val Loss: 4.255698,  Acc: 0.057100
Took 0.73 minutes.

Epoch: 2
Train Loss: 0.000104, Acc: 0.073375
Val Loss: 4.073224,  Acc: 0.092900
Took 0.69 minutes.

Epoch: 3
Train Loss: 0.000100, Acc: 0.105500
Val Loss: 3.990079,  Acc: 0.107400
Took 0.70 minutes.

Epoch: 4
Train Loss: 0.000097, Acc: 0.134050
Val Loss: 3.834288,  Acc: 0.143500
Took 0.70 minutes.

Epoch: 5
Train Loss: 0.000093, Acc: 0.165300
Val Loss: 3.664247,  Acc: 0.183200
Took 0.70 minutes.

Epoch: 6
Train Loss: 0.000090, Acc: 0.190525
Val Loss: 3.604045,  Acc: 0.194900
Took 0.70 minutes.

Epoch: 7
Train Loss: 0.000087, Acc: 0.216125
Val Loss: 3.482636,  Acc: 0.232800
Took 0.70 minutes.

Epoch: 8
Train Loss: 0.000083, Acc: 0.253675
Val Loss: 3.359976,  Acc: 0.264100
Took 0.70 minutes.

Epoch: 9
Train Loss: 0.000080, Acc: 0.288250
Val Loss: 3.454177,  Acc: 0.252000
Took 0.69 minutes.

Epoch: 10
Train Loss: 0.000077, Acc: 0.316625
Val Loss: 3.1902

In [ ]:
import numpy as np
import time

# Directory for saving model weights during training
weightsDir = Path('models/weights')
weightsDir.mkdir(exist_ok=True)
resultsDir = Path('results')
resultsDir.mkdir(exist_ok=True)

# Settings for training run
dataset = "cifar100"
noise_rate = 0.2
noise_mode="sym"
batch_size=128
datapath=datapath
noise_file = "dense20pctSymCifar100"
epochs = 200
weightFileName = f"R34_10Sprase_Cifar100_{int(noise_rate*100)}pct_{noise_mode}"
sparsity = 0.9

f = open(resultsDir / Path(weightFileName + '.txt'), 'w')

# intialise device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# get data loaders for training, validation, and test sets
dataLoader = CifarDataloader(dataset=dataset, noise_rate=noise_rate, noise_mode=noise_mode, batch_size=batch_size, datapath=datapath, noise_file = noise_file)
train_loader = dataLoader.trainLoader
val_loader = dataLoader.validLoader
test_loader = dataLoader.testLoader
n = len(train_loader.dataset)

# initialise model and learning rate schedule
model, optimizer = get_model(dataset, lr=0.1)
schedule = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2) # need to adjust this schedule to match a paper
model = model.to(device)

# save the model for best validaiton loss throughout training    
best_epoch, best_val_loss = 0, np.inf
train_loss_list, train_acc_list = [], []
val_loss_list, val_acc_list = [], []

for epoch in range(epochs):

    start = time.time()
    print("Epoch: {}".format(epoch+1))
    f.write("Epoch: {}\n".format(epoch+1))

    train_loss, train_acc = train(model, optimizer, train_loader, device)
    train_loss /= n
    print("Train Loss: {:.6f}, Acc: {:.6f}".format(train_loss, train_acc))
    f.write("Train Loss: {:.6f}, Acc: {:.6f}\n".format(train_loss, train_acc))

    val_loss, val_acc = evaluate(model, val_loader, device)
    print("Val Loss: {:.6f},  Acc: {:.6f}".format(val_loss, val_acc))
    f.write("Val Loss: {:.6f},  Acc: {:.6f}\n".format(val_loss, val_acc))

    if val_loss < best_val_loss:
        best_epoch = epoch
        best_val_loss = val_loss
        best_train_loss = train_loss
        torch.save(model.state_dict(), weightsDir / Path(f'{weightFileName}_best_forward.pkl'))

    schedule.step()

    train_loss_list.append(train_loss)
    train_acc_list.append(train_acc)
    val_loss_list.append(val_loss)
    val_acc_list.append(val_acc)

    end = time.time()
    print("Took {:.2f} minutes.\n".format((end - start) / 60))
    f.write("Took {:.2f} minutes.\n".format((end - start) / 60))

print("Best epoch:", best_epoch+1)
f.write(f"Best epoch: {best_epoch+1}\n")
print("Final train loss: {:.4f}".format(train_loss_list[best_epoch]))
f.write("Final train loss: {:.4f}\n".format(train_loss_list[best_epoch]))
print("Final train accuracy: {:.4f}\n".format(train_acc_list[best_epoch]))
f.write("Final train accuracy: {:.4f}\n".format(train_acc_list[best_epoch]))
print("Final val loss: {:.4f}".format(val_loss_list[best_epoch]))
f.write("Final val loss: {:.4f}\n".format(val_loss_list[best_epoch]))
print("Final val accuracy: {:.4f}\n".format(val_acc_list[best_epoch]))
f.write("Final val accuracy: {:.4f}\n".format(val_acc_list[best_epoch]))

# load best model saved during training
model.load_state_dict(torch.load(weightsDir / Path(f'{weightFileName}_best_forward.pkl'), map_location=device))
test_loss, test_acc = evaluate(model, test_loader, device)
print("Test accuracy: {:.4f}".format(test_acc))
f.write("Test accuracy: {:.4f}\n".format(test_acc))

f.close()

Train Loss: 0.000093, Acc: 0.167850
Val Loss: 3.743893,  Acc: 0.172600
Took 0.70 minutes.

Epoch: 6
Train Loss: 0.000090, Acc: 0.195425
Val Loss: 3.614808,  Acc: 0.200000
Took 0.70 minutes.

Epoch: 7
Train Loss: 0.000087, Acc: 0.222300
Val Loss: 3.575471,  Acc: 0.209200
Took 0.70 minutes.

Epoch: 8
Train Loss: 0.000077, Acc: 0.308375
Val Loss: 3.186904,  Acc: 0.304200
Took 0.70 minutes.

Epoch: 11
Train Loss: 0.000075, Acc: 0.327300
Val Loss: 3.194636,  Acc: 0.306000
Took 0.69 minutes.

Epoch: 12
Train Loss: 0.000073, Acc: 0.352975
Val Loss: 3.054210,  Acc: 0.334100
Took 0.70 minutes.

Epoch: 13
Train Loss: 0.000070, Acc: 0.378550
Val Loss: 3.047481,  Acc: 0.344600
Took 0.70 minutes.

Epoch: 14
Train Loss: 0.000068, Acc: 0.398475
Val Loss: 3.017823,  Acc: 0.351200
Took 0.70 minutes.

Epoch: 15
Train Loss: 0.000066, Acc: 0.419600
Val Loss: 2.982847,  Acc: 0.370400
Took 0.70 minutes.

Epoch: 16
Train Loss: 0.000064, Acc: 0.433525
Val Loss: 2.942379,  Acc: 0.380500
Took 0.70 minutes.

Epo

In [ ]:
import numpy as np
import time

# Directory for saving model weights during training
weightsDir = Path('models/weights')
weightsDir.mkdir(exist_ok=True)
resultsDir = Path('results')
resultsDir.mkdir(exist_ok=True)

# Settings for training run
dataset = "cifar100"
noise_rate = 0.2
noise_mode="sym"
batch_size=128
datapath=datapath
noise_file = "dense20pctSymCifar100"
epochs = 200
weightFileName = f"R34_30Sprase_Cifar100_{int(noise_rate*100)}pct_{noise_mode}"
sparsity = 0.7

f = open(resultsDir / Path(weightFileName + '.txt'), 'w')

# intialise device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# get data loaders for training, validation, and test sets
dataLoader = CifarDataloader(dataset=dataset, noise_rate=noise_rate, noise_mode=noise_mode, batch_size=batch_size, datapath=datapath, noise_file = noise_file)
train_loader = dataLoader.trainLoader
val_loader = dataLoader.validLoader
test_loader = dataLoader.testLoader
n = len(train_loader.dataset)

# initialise model and learning rate schedule
model, optimizer = get_model(dataset, lr=0.1)
schedule = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2) # need to adjust this schedule to match a paper
model = model.to(device)

# save the model for best validaiton loss throughout training    
best_epoch, best_val_loss = 0, np.inf
train_loss_list, train_acc_list = [], []
val_loss_list, val_acc_list = [], []

for epoch in range(epochs):

    start = time.time()
    print("Epoch: {}".format(epoch+1))
    f.write("Epoch: {}\n".format(epoch+1))

    train_loss, train_acc = train(model, optimizer, train_loader, device)
    train_loss /= n
    print("Train Loss: {:.6f}, Acc: {:.6f}".format(train_loss, train_acc))
    f.write("Train Loss: {:.6f}, Acc: {:.6f}\n".format(train_loss, train_acc))

    val_loss, val_acc = evaluate(model, val_loader, device)
    print("Val Loss: {:.6f},  Acc: {:.6f}".format(val_loss, val_acc))
    f.write("Val Loss: {:.6f},  Acc: {:.6f}\n".format(val_loss, val_acc))

    if val_loss < best_val_loss:
        best_epoch = epoch
        best_val_loss = val_loss
        best_train_loss = train_loss
        torch.save(model.state_dict(), weightsDir / Path(f'{weightFileName}_best_forward.pkl'))

    schedule.step()

    train_loss_list.append(train_loss)
    train_acc_list.append(train_acc)
    val_loss_list.append(val_loss)
    val_acc_list.append(val_acc)

    end = time.time()
    print("Took {:.2f} minutes.\n".format((end - start) / 60))
    f.write("Took {:.2f} minutes.\n".format((end - start) / 60))

print("Best epoch:", best_epoch+1)
f.write(f"Best epoch: {best_epoch+1}\n")
print("Final train loss: {:.4f}".format(train_loss_list[best_epoch]))
f.write("Final train loss: {:.4f}\n".format(train_loss_list[best_epoch]))
print("Final train accuracy: {:.4f}\n".format(train_acc_list[best_epoch]))
f.write("Final train accuracy: {:.4f}\n".format(train_acc_list[best_epoch]))
print("Final val loss: {:.4f}".format(val_loss_list[best_epoch]))
f.write("Final val loss: {:.4f}\n".format(val_loss_list[best_epoch]))
print("Final val accuracy: {:.4f}\n".format(val_acc_list[best_epoch]))
f.write("Final val accuracy: {:.4f}\n".format(val_acc_list[best_epoch]))

# load best model saved during training
model.load_state_dict(torch.load(weightsDir / Path(f'{weightFileName}_best_forward.pkl'), map_location=device))
test_loss, test_acc = evaluate(model, test_loader, device)
print("Test accuracy: {:.4f}".format(test_acc))
f.write("Test accuracy: {:.4f}\n".format(test_acc))

f.close()

Files already downloaded and verified
Epoch: 1
Train Loss: 0.000112, Acc: 0.037175
Val Loss: 4.231076,  Acc: 0.060500
Took 0.72 minutes.

Epoch: 2
Train Loss: 0.000104, Acc: 0.073325
Val Loss: 4.089241,  Acc: 0.084100
Took 0.68 minutes.

Epoch: 3
Train Loss: 0.000100, Acc: 0.103625
Val Loss: 3.899287,  Acc: 0.127300
Took 0.68 minutes.

Epoch: 4
Train Loss: 0.000096, Acc: 0.135425
Val Loss: 3.870545,  Acc: 0.145900
Took 0.69 minutes.

Epoch: 5
Train Loss: 0.000093, Acc: 0.166575
Val Loss: 3.693165,  Acc: 0.173700
Took 0.69 minutes.

Epoch: 6
Train Loss: 0.000090, Acc: 0.193050
Val Loss: 3.641437,  Acc: 0.198000
Took 0.69 minutes.

Epoch: 7
Train Loss: 0.000087, Acc: 0.222025
Val Loss: 3.504777,  Acc: 0.224100
Took 0.69 minutes.

Epoch: 8
Train Loss: 0.000084, Acc: 0.251000
Val Loss: 3.427900,  Acc: 0.241900
Took 0.69 minutes.

Epoch: 9
Train Loss: 0.000081, Acc: 0.271850
Val Loss: 3.373901,  Acc: 0.256600
Took 0.68 minutes.

Epoch: 10
Train Loss: 0.000078, Acc: 0.307850
Val Loss: 3.2314

In [ ]:
import numpy as np
import time

# Directory for saving model weights during training
weightsDir = Path('models/weights')
weightsDir.mkdir(exist_ok=True)
resultsDir = Path('results')
resultsDir.mkdir(exist_ok=True)

# Settings for training run
dataset = "cifar100"
noise_rate = 0.2
noise_mode="sym"
batch_size=128
datapath=datapath
noise_file = "dense20pctSymCifar100"
epochs = 200
weightFileName = f"R34_40Sprase_Cifar100_{int(noise_rate*100)}pct_{noise_mode}"
sparsity = 0.6

f = open(resultsDir / Path(weightFileName + '.txt'), 'w')

# intialise device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# get data loaders for training, validation, and test sets
dataLoader = CifarDataloader(dataset=dataset, noise_rate=noise_rate, noise_mode=noise_mode, batch_size=batch_size, datapath=datapath, noise_file = noise_file)
train_loader = dataLoader.trainLoader
val_loader = dataLoader.validLoader
test_loader = dataLoader.testLoader
n = len(train_loader.dataset)

# initialise model and learning rate schedule
model, optimizer = get_model(dataset, lr=0.1)
schedule = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2) # need to adjust this schedule to match a paper
model = model.to(device)

# save the model for best validaiton loss throughout training    
best_epoch, best_val_loss = 0, np.inf
train_loss_list, train_acc_list = [], []
val_loss_list, val_acc_list = [], []

for epoch in range(epochs):

    start = time.time()
    print("Epoch: {}".format(epoch+1))
    f.write("Epoch: {}\n".format(epoch+1))

    train_loss, train_acc = train(model, optimizer, train_loader, device)
    train_loss /= n
    print("Train Loss: {:.6f}, Acc: {:.6f}".format(train_loss, train_acc))
    f.write("Train Loss: {:.6f}, Acc: {:.6f}\n".format(train_loss, train_acc))

    val_loss, val_acc = evaluate(model, val_loader, device)
    print("Val Loss: {:.6f},  Acc: {:.6f}".format(val_loss, val_acc))
    f.write("Val Loss: {:.6f},  Acc: {:.6f}\n".format(val_loss, val_acc))

    if val_loss < best_val_loss:
        best_epoch = epoch
        best_val_loss = val_loss
        best_train_loss = train_loss
        torch.save(model.state_dict(), weightsDir / Path(f'{weightFileName}_best_forward.pkl'))

    schedule.step()

    train_loss_list.append(train_loss)
    train_acc_list.append(train_acc)
    val_loss_list.append(val_loss)
    val_acc_list.append(val_acc)

    end = time.time()
    print("Took {:.2f} minutes.\n".format((end - start) / 60))
    f.write("Took {:.2f} minutes.\n".format((end - start) / 60))

print("Best epoch:", best_epoch+1)
f.write(f"Best epoch: {best_epoch+1}\n")
print("Final train loss: {:.4f}".format(train_loss_list[best_epoch]))
f.write("Final train loss: {:.4f}\n".format(train_loss_list[best_epoch]))
print("Final train accuracy: {:.4f}\n".format(train_acc_list[best_epoch]))
f.write("Final train accuracy: {:.4f}\n".format(train_acc_list[best_epoch]))
print("Final val loss: {:.4f}".format(val_loss_list[best_epoch]))
f.write("Final val loss: {:.4f}\n".format(val_loss_list[best_epoch]))
print("Final val accuracy: {:.4f}\n".format(val_acc_list[best_epoch]))
f.write("Final val accuracy: {:.4f}\n".format(val_acc_list[best_epoch]))

# load best model saved during training
model.load_state_dict(torch.load(weightsDir / Path(f'{weightFileName}_best_forward.pkl'), map_location=device))
test_loss, test_acc = evaluate(model, test_loader, device)
print("Test accuracy: {:.4f}".format(test_acc))
f.write("Test accuracy: {:.4f}\n".format(test_acc))

f.close()